In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 10
    self.test_batch = 1000
    self.global_epochs = 100
    self.local_epochs = 2
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 100
    self.worker_num = 20
    self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
lr = 0

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for i in range(len(dataset)):
      self.data.append(dataset[i][0][0])
      self.target.append(dataset[i][1][0])

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_shakespeare.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_shakespeare.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[28, 6, 70, 62, 57, 35, 26, 139, 22, 108, 8, 7, 23, 55, 59, 129, 50, 107, 56, 114]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class RNN1(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN1, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN2(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN2, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN3(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN3, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=3, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output
    
class RNN4(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN4, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=4, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

In [15]:
class Server():
  def __init__(self):
    self.global_model = RNN2()
 

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.global_model = copy.deepcopy(self.global_model)


  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.global_model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
      del worker.global_model 
    self.global_model.load_state_dict(new_params)

In [16]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    if len(trainset)<=2 :
      trainset = [trainset,copy.deepcopy(trainset),copy.deepcopy(trainset)]
    else:
      split_len = [(len(trainset) + i) // 3 for i in range(3)]
      trainset = torch.utils.data.random_split(trainset,split_len)
    self.trainloader_local = torch.utils.data.DataLoader(trainset[0],batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.trainloader_lambda = torch.utils.data.DataLoader(trainset[1],batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.trainloader_global = torch.utils.data.DataLoader(trainset[2],batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.global_model = None
    self.local_model = RNN2()
    self.mix_model = None
    self.lmd = None
    self.aggregation_weight = None


  def local_train(self):
    loss = float('inf')
    self.mix_model = RNN2()
    model_tmp = RNN2()
    for lmd in np.arange(0.0,1.1,0.1):
      new_params = OrderedDict()
      global_state = self.global_model.state_dict()
      local_state = self.local_model.state_dict()
      for key in global_state.keys():
        new_params[key] = lmd*local_state[key] + (1.0-lmd)*global_state[key]     
      model_tmp.load_state_dict(new_params)
      model_tmp = model_tmp.to(args.device)
      if lmd!=0.0:
        _,_ = train(model_tmp,args.criterion,self.trainloader_local,args.local_epochs)

      _,tmp = test(model_tmp,args.criterion,self.trainloader_lambda)
      if tmp<loss:
        loss = tmp
        self.lmd = lmd
        self.mix_model = copy.deepcopy(model_tmp)

    if self.lmd!=0.0:
      self.mix_model = self.mix_model.to('cpu')
      new_params = OrderedDict()
      mix_state = self.mix_model.state_dict()
      global_state = self.global_model.state_dict()
      for key in global_state.keys():
        new_params[key] = (mix_state[key] - (1.0-self.lmd)*global_state[key])/self.lmd    
      self.local_model.load_state_dict(new_params)
      self.mix_model = self.mix_model.to(args.device)
    
    model_tmp = model_tmp.to('cpu')
    del model_tmp


  def global_train(self):
    model_tmp = copy.deepcopy(self.mix_model)
    model_tmp = model_tmp.to('cpu')
    acc_train,loss_train = train(self.mix_model,args.criterion,self.trainloader_global,args.local_epochs)
    acc_valid,loss_valid = test(self.mix_model,args.criterion,self.valloader)
    self.mix_model = self.mix_model.to('cpu')
    new_params = OrderedDict()
    new_state = self.mix_model.state_dict()
    old_state = model_tmp.state_dict()
    global_state = self.global_model.state_dict()
    for key in new_state.keys():
        new_params[key] = global_state[key]+new_state[key]-old_state[key]  
    self.global_model.load_state_dict(new_params)
    del self.mix_model
    del model_tmp
    return acc_train,loss_train,acc_valid,loss_valid


  def test_step(self):
    self.local_train()
    mix_model = RNN2()
    new_params = OrderedDict()
    global_state = self.global_model.state_dict()
    local_state = self.local_model.state_dict()
    for key in global_state.keys():
      new_params[key] = self.lmd*local_state[key] + (1.0-self.lmd)*global_state[key]     
    mix_model.load_state_dict(new_params)
    mix_model = mix_model.to(args.device)
    acc,loss = test(mix_model,args.criterion,self.testloader)
    return acc,loss

In [17]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)



In [18]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [19]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [20]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_model(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    worker.local_train()
    acc_train_tmp,loss_train_tmp,acc_valid_tmp,loss_valid_tmp = worker.global_train()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_model(sample_worker)
  '''
  server.model.to(args.device)
  for worker in workers:
    acc_valid_tmp,loss_valid_tmp = test(server.model,args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  server.model.to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:582: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/aten/src/ATen/native/cudnn/RNN.cpp:775.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


Epoch1  loss:4.236473634839058  accuracy:8.069107456090777
Epoch2  loss:4.174220880203776  accuracy:12.53862684942038
Epoch3  loss:4.118661376171642  accuracy:13.001937767141902
Epoch4  loss:4.100005786617597  accuracy:12.997982070939367
Epoch5  loss:4.082577646772067  accuracy:10.234808034876503
Epoch6  loss:4.064220839407708  accuracy:11.202437676864749
Epoch7  loss:4.0475626581245  accuracy:11.73309805422324
Epoch8  loss:4.031977803839578  accuracy:12.732748230263683
Epoch9  loss:4.016399679912461  accuracy:13.025594142213368
Epoch10  loss:4.002757818831339  accuracy:13.261339756248455
Epoch11  loss:3.9879814263847138  accuracy:13.867955902400732
Epoch12  loss:3.9763136920001765  accuracy:13.564641450471768
Epoch13  loss:3.9631829420725495  accuracy:13.613631040183954
Epoch14  loss:3.95337160759502  accuracy:13.660783520281932
Epoch15  loss:3.9424619830316967  accuracy:13.44790857503284
Epoch16  loss:3.9324807276328406  accuracy:13.365174853865257
Epoch17  loss:3.9228401571512235  a

In [22]:
acc_test = []
loss_test = []

nums = 0
for worker in workers:
  nums += worker.test_data_num

start = time.time()

for i,worker in enumerate(workers):
  worker.aggregation_weight = 1.0*worker.test_data_num/nums
  worker.global_model = copy.deepcopy(server.global_model)
  worker.local_train()
  acc_tmp,loss_tmp = test(worker.mix_model,args.criterion,worker.testloader)
  acc_test.append(acc_tmp)
  loss_test.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
  worker.mix_model = worker.mix_model.to('cpu')
  del worker.mix_model,worker.global_model

end = time.time()

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 accuracy:37.883908890521674  loss:2.259577138083322
Worker2 accuracy:19.243986254295532  loss:3.3030569553375244
Worker3 accuracy:11.363636363636363  loss:4.129376411437988
Worker4 accuracy:19.696969696969695  loss:3.8436367511749268
Worker5 accuracy:0.0  loss:4.1975603103637695
Worker6 accuracy:4.761904761904762  loss:4.479913234710693
Worker7 accuracy:20.0  loss:4.857960224151611
Worker8 accuracy:20.66115702479339  loss:3.1175718307495117
Worker9 accuracy:33.333333333333336  loss:4.348940372467041
Worker10 accuracy:15.725806451612904  loss:3.228642225265503
Worker11 accuracy:24.299065420560748  loss:3.164780855178833
Worker12 accuracy:17.5  loss:4.214111804962158
Worker13 accuracy:31.110254433307635  loss:2.554711182912191
Worker14 accuracy:15.868263473053892  loss:3.123332977294922
Worker15 accuracy:12.5  loss:4.019942283630371
Worker16 accuracy:29.682090022033364  loss:2.445144534111023
Worker17 accuracy:19.23076923076923  loss:3.732733726501465
Worker18 accuracy:22.9729729